In [1]:
from PyQt5.QtWidgets import *
from PyQt5 import uic
import sys
import os
import math
from PyQt5.QtGui import QPixmap
from PyQt5.QtGui import QStandardItemModel
from PyQt5.QtGui import QStandardItem
from PyQt5 import QtGui
import matplotlib.image as img 
import matplotlib.pyplot as plt
import pytesseract
import cv2
import os
from skimage import io
from PIL import Image  
import numpy as np
import re
import pandas as pd

In [2]:
def filter_data(text_r,data_base):
    data_filter_result=[]
    data_filter_label=[]

    for text in text_r:
        sig=0
        label_index=0
        a=0

        for i in range(len(data_base.index)):
            for data in data_base.iloc[i]:
                if data==text:
                    sig=1
                    label_index=i

        if sig ==1:
            data_filter_label.append(data_base.index[label_index])
        else:
            data_filter_label.append('영향없음')
        data_filter_result.append(text)
    data_filter_result=pd.Series(data_filter_result)
    data_filter_label=pd.Series(data_filter_label)
    df_result=pd.concat([data_filter_result,data_filter_label],axis=1)
    return(df_result)

In [3]:
def text_parse(text):
    text_result=[]
    try:
        text=text.replace(" ","")
    except:
        text="".join(text)
        text=text.replace(" ","")
    text=text.replace('\n',"")
    text=text.strip()
    text=list(text)
    text_pre_1=[]
    sig=0
    text_sig=0

    for i in range(len(text)):
        if ((text[i-1]=='성')&(text[i]=='분')):
            sig=1
        if sig==1:
            text_pre_1.append(text[i])
    text_pre_1="".join(text_pre_1)
    #print('성분 Parsing :\n',text_pre_1)
    
    
    p=re.compile('\w+,') #정규식 
    text_pre_2=p.findall(text_pre_1)
    text_pre_2=list(text_pre_2)

    
    for i in range(len(text_pre_2)):
        text_pre_2[i]=text_pre_2[i].replace(',',"") #쉼표 지우기 
        #print(text_pre_2[i])
        text_result.append(text_pre_2[i])
    #print(text_pre_2)
    return(text_result)

In [4]:
# 한번 작동하고 실행창 끄면 커널이 죽음 커널 재시작을 해야함

ui_file = 'C:/Users/qjawl/ocr.ui'

class MainDialog(QDialog):
    def __init__(self):
        QDialog.__init__(self, None)
        uic.loadUi(ui_file, self)
        self.pushButton_2.clicked.connect(self.load_data)
        self.pushButton.clicked.connect(self.ocr)
        self.listWidget.itemDoubleClicked.connect(self.find_label)
        self.resultData=[]
        self.listWidget.currentItemChanged.connect(self.change_list)
        self.pushButton_3.clicked.connect(self.find_u2)
        
        self.data_all=pd.read_excel('C:/Users/qjawl/Desktop/창융/화장품데이터/상세데이터.xlsx')
    def load_data(self):
        path="C:/Users/qjawl/Desktop/대외활동/창의융합캡스톤/cosmetics/cosmetics/images/ocr/ocr_1.jpg"
        
       
        self.pixmap=QPixmap()
        self.pixmap.load(path)
        self.lbl_picture.setPixmap(self.pixmap)
    
    
        
        
    def ocr(self):
        self.textBrowser.clear()
        self.textBrowser_2.clear()
        self.listWidget.clear()
        path="C:/Users/qjawl/Desktop/대외활동/창의융합캡스톤/cosmetics/cosmetics/images/ocr/ocr_1.jpg"
        img = io.imread(path)
        img=cv2.resize(img,None,fx=1.3,fy=1.3) 
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #print('gray 이미지 shape :',gray.shape)
        filename = "{}.png".format(os.getpid())
        cv2.imwrite(filename, gray)
        gray_scale_img=io.imread(filename)
        pil_image=Image.fromarray(gray_scale_img)
        #pil_image.show()
        gray=cv2.threshold(gray,127,255,cv2.THRESH_TRUNC)[1]
        filename = "{}.png".format(os.getpid())
        cv2.imwrite(filename, gray)
        gray_scale_img=io.imread(filename)
        pil_image=Image.fromarray(gray_scale_img)
        #pil_image.show()
        pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe' 
        text = pytesseract.image_to_string(Image.open(filename), lang = 'kor')
        os.remove(filename)
        text_r=text_parse(text)
        data_base=pd.read_excel('C:/Users/qjawl/Desktop/창융/화장품데이터/화장품.xlsx')
        data_base=data_base.T
        self.result_data=filter_data(text_r,data_base)
        result_data_name=self.result_data.iloc[:,0]
        result_data_label=self.result_data.iloc[:,1]
        
   
         
        for x in result_data_name:
            self.listWidget.addItem(x)
            
  
    def find_u2(self):
        self.textBrowser.clear()
        self.textBrowser_2.clear()
        self.listWidget.clear()
        
        a=self.result_data[self.result_data.iloc[:,1]!="영향없음"]
        result_data_name=a.iloc[:,0]
        
        for x in result_data_name:
            self.listWidget.addItem(x)
        
        
        
        
        
    def find_label(self):
        item=self.listWidget.currentItem().text()
        a=self.result_data[self.result_data.iloc[:,0]==item]
        b=self.data_all[self.result_data.iloc[:,0]==item]
        
        item_label=a[1]
        item_label=list(item_label)
        
        item_label_b=b.iloc[:,2]
        try:
            item_label=item_label[0]
        except:
            pass
        self.textBrowser.append(item_label)
        
        try:
           # item_label_b=item_label_b[0]
            pass
        except:
            pass
        for test in item_label_b:
            self.textBrowser_2.append(test)
        
    def change_list(self):
        self.textBrowser.clear()
        self.textBrowser_2.clear()    
        
QApplication.setStyle("fusion")        
app = QApplication(sys.argv)
main_dialog = MainDialog()
main_dialog.show()
sys.exit(app.exec_())


SystemExit: 0

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
